## Importing Library

In [38]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [39]:
import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn

## Importing Dataset

In [40]:
df = pd.read_csv("movies_metadata.csv",low_memory = False)

## Preprocessing

In [41]:
df = df.loc[df["original_language"]=="en"]
df = df.loc[df["genres"]!="[]"]
df = df.loc[df['vote_count'] >= 10]
df1 = df[["genres","id","imdb_id","original_language","production_companies","title"]]
df.shape

(16676, 24)

### Extracting Genre

In [42]:
index=0
for i in df1["genres"]:
    dist = eval(i)
    genres=[]
    for j in dist:
        genres.append(j['name'])
    x = " ".join(genres)
    df1.iloc[index]['genres'] = x
    index+=1
df1.reset_index(drop = True, inplace=True)

### Extracting Production company

In [43]:
df1["production_companies"] = df1["production_companies"].astype(str) 
index=0
for i in df1["production_companies"]:
    dist = eval(i)
    prod=[]
    for j in dist:
        prod.append(j['name'])
    x = " ".join(prod)
    df1.iloc[index]['production_companies'] = x
    index+=1

### Combining Different Feature into a string

In [44]:
features = ["genres","production_companies","title"]
def combine_features(row):
    return row['title'] + " "  + row['genres'] + " " + row['production_companies'] 

df1["combined_features"] = df1.apply(combine_features,axis = 1)

In [45]:
df1

,genres,id,imdb_id,original_language,production_companies,title,combined_features
0,Animation Comedy Family,862,tt0114709,en,Pixar Animation Studios,Toy Story,Toy Story Animation Comedy Family Pixar Animat...
1,Adventure Fantasy Family,8844,tt0113497,en,TriStar Pictures Teitler Film Interscope Commu...,Jumanji,Jumanji Adventure Fantasy Family TriStar Pictu...
2,Romance Comedy,15602,tt0113228,en,Warner Bros. Lancaster Gate,Grumpier Old Men,Grumpier Old Men Romance Comedy Warner Bros. L...
3,Comedy Drama Romance,31357,tt0114885,en,Twentieth Century Fox Film Corporation,Waiting to Exhale,Waiting to Exhale Comedy Drama Romance Twentie...
4,Comedy,11862,tt0113041,en,Sandollar Productions Touchstone Pictures,Father of the Bride Part II,Father of the Bride Part II Comedy Sandollar P...
...,...,...,...,...,...,...,...
16671,Family Animation Romance Comedy,455661,tt6969946,en,Ringling College of Art and Design,In a Heartbeat,In a Heartbeat Family Animation Romance Comedy...
16672,Drama Family TV Movie,39562,tt0889600,en,,To Be Fat Like Me,To Be Fat Like Me Drama Family TV Movie
16673,Comedy,14008,tt0294425,en,Walt Disney Television Disney Channel Leider/S...,Cadet Kelly,Cadet Kelly Comedy Walt Disney Television Disn...
16674,Fantasy Comedy,49271,tt0127948,en,Star Film Company,The Devilish Tenant,The Devilish Tenant Fantasy Comedy Star Film C...


## Training

In [46]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df1["combined_features"])
similarity_scores = cosine_similarity(count_matrix)

## Predicting

In [47]:
def Categorical(movie,no_of_recommendation):
    dim = df1.shape
    index = -1
    for i in range(dim[0]):
        if df1.loc[i]['title']==movie:
            index = i
            break
            
    if(index == -1):
        print("Movie not present")
        return
    
    similarity_movies = similarity_scores[index]
    
    dist = {}
    for i in range(dim[0]):
        dist[i] = similarity_movies[i]
    max_dist = {key:val for key,val in sorted(dist.items(),key = lambda ele:ele[1],reverse = True)}
    final_list = list(max_dist.keys())
    
    i=0
    recommendation = []
    while(i<no_of_recommendation):
        recommendation.append(df1.loc[final_list[i]]['title'])
        i+=1
    return recommendation

In [48]:
recommendation = Categorical("Batman Begins",8)
for i in recommendation:
    print(i)

Batman Begins
The Dark Knight
The Dark Knight Rises
The Losers
RED 2
Suicide Squad
Batman: The Killing Joke
DC Showcase: Catwoman


In [49]:
recommendation = Categorical("Jumanji",8)
for i in recommendation:
    print(i)

Jumanji
Zathura: A Space Adventure
Hook
Kazaam
Peter Pan
Bingo
Operation Dumbo Drop
Magic in the Water
